In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from scipy.stats import multivariate_normal
from scipy.stats import norm

import statsmodels as sm
import statsmodels.tsa.api as tsa
from statsmodels.tsa.base.datetools import dates_from_str
import statsmodels.formula.api as smf

colors = sns.color_palette()
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [31]:
# Example (but real) data from the statsmodel database
mdata = sm.datasets.macrodata.load_pandas().data

# prepare the dates index
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)

mdata = mdata[['realgdp','realcons','realinv']]
mdata.index = pd.DatetimeIndex(quarterly)
data = np.log(mdata).diff().dropna()
data = data.iloc[:, 1:]

In [32]:
data.shape

(202, 2)

In [33]:
true_lag = 1

model = tsa.VAR(data)
results = model.fit(true_lag)

In [34]:
M = len(results.names)
L = results.k_ar
mu = results.intercept
A = results.coefs    

error = np.asarray(results.resid)
T = error.shape[0]
Sigma = (error.T @ error)/T

In [35]:
def stationary_dist(mu, A, Sigma):

    M, L = A.shape[2], A.shape[0] 
    K = M*L
    
    mu_comp = np.zeros((K, 1))
    mu_comp[:M, 0] = mu
    A_row = np.hstack([A[i, :, :] for i in range(L)])
    A_comp = np.vstack([A_row, 
                        np.hstack([np.eye(M*(L-1)), np.zeros((M*(L-1), M))])])
    Sigma_comp = np.zeros((M*L, M*L))
    Sigma_comp[:M, :M] = Sigma

    mu_stationary = np.linalg.solve(np.eye(K) - A_comp, mu_comp)
    Sigma_stationary = sp.linalg.solve_discrete_lyapunov(A_comp, Sigma_comp)

    return mu_stationary, Sigma_stationary

In [113]:
# data generating process
def true_model(N, mu, A, Sigma):
    '''Simulating the true model'''
    
    M, L = A.shape[2], A.shape[0] 
    K = M*L
    
    mu_stationary, Sigma_stationary = stationary_dist(mu, A, Sigma)
        
    initial_x = multivariate_normal(mu_stationary.squeeze(), Sigma_stationary).rvs()
    shocks = multivariate_normal(np.zeros(len(mu)), Sigma)
    error = shocks.rvs(N - L).T
    
    X = np.zeros((M, N))
    X[:, :L] = initial_x.reshape(L, M).T
    
    for t in range(N - L):
        AX = np.zeros((M, 1))
        for lag in range(L):
            AX += A[lag, :, :] @ X[:, t + L - 1 - lag].reshape(M, 1)
        X[:, L + t] = (mu.reshape(M, 1) + AX + error[:, t].reshape(M, 1)).squeeze()
    
    return pd.DataFrame(data = X.T, index = data.index[-N:]), error


In [125]:
simul, error_y = true_model(T, mu, A, Sigma)

In [126]:
simul = simul.sub(simul.mean(), axis='columns')

In [127]:
Y_path = np.asarray(simul)

In [128]:
Y_data = np.hstack([np.asarray(simul.iloc[1:, :]), np.asarray(simul.iloc[:-1, :])])

## Generate a latent path

In [129]:
rho = .7
sigma = .01

x_path = []

x0 = (sigma/np.sqrt(1.0 - rho*rho))*norm.rvs()
x_path.append(x0)

error_x = sigma*norm.rvs(size=200)

for t in range(200):
    x_path.append(rho*x_path[t] + error_x[t])
X_path = np.asarray(x_path).reshape(201, 1)

In [137]:
shocks = np.vstack([error_y, error_x.reshape(1, 200)])

## Generate returns

In [149]:
nu = np.asarray([[0.02, 0.01, 0.04, 0.0, -0.02, 0.02, 0.04, 0.005]]).T
alpha = np.asarray([[.01, .05, .1],
                    [.02, .1, .0],
                    [-.3, -.2, .05],
                    [.006, .02, .12],
                    [.03, -.1, .03],
                    [-.3, -.2, .05],
                    [.03, .02, .09],
                    [-.02, .01, .06]])

In [151]:
logR = nu - np.diag(alpha @ alpha.T).reshape(8, 1)/2 + alpha @ shocks

In [159]:
simul_data = np.hstack([Y_data, logR.T])

In [168]:
np.savetxt('../data/data.dat', simul_data, fmt='% 2.10f', delimiter=' ')

In [171]:
np.savetxt('../data/initial_particle.dat', X_path, fmt='% 2.10f', delimiter=' ')

In [172]:
A

array([[[ 0.2485817218,  0.0251747264],
        [ 3.2743679974,  0.0136133571]]])

In [174]:
np.linalg.cholesky(Sigma)

array([[ 0.0065171165,  0.          ],
       [ 0.0061470262,  0.0400707285]])